In [7]:
import discord
import random
import os
import asyncio

import pandas as pd
import requests

#########################

import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
df_all= pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
## bowler stats

bowler_stats = df_all.groupby(['bowler','bowling_team']).agg(   ##,'innings'
    num_innings = ('match_id','nunique'),
    runs = ('runs_conceeded','sum'),
    balls = ('islegal' ,'sum'),
    wkts = ('isBowlerWicket','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum'),
    wides = ('wides','sum'),
    noballs = ('noballs','sum')
        
    
).reset_index()

bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']
bowler_stats['strike_rate'] = bowler_stats['balls']/bowler_stats['wkts']
bowler_stats['bpb'] = bowler_stats['balls']/(bowler_stats['fours']+bowler_stats['sixes'])
bowler_stats['dot_%'] = 100*bowler_stats['dots']/bowler_stats['balls']

#########ADDING 3+,4+,5+ wkts#########
mbm_bowl = df_all.groupby(['bowler','match_id'])['isBowlerWicket'].sum().reset_index().sort_values(by='bowler')

_3plus = mbm_bowl[mbm_bowl.isBowlerWicket>=3].groupby('bowler')['match_id'].count().reset_index()
_4plus = mbm_bowl[mbm_bowl.isBowlerWicket>=4].groupby('bowler')['match_id'].count().reset_index()
_5plus = mbm_bowl[mbm_bowl.isBowlerWicket>=5].groupby('bowler')['match_id'].count().reset_index()

_3plus.columns = ['bowler','3+_wkts']
_4plus.columns = ['bowler','4+_wkts']
_5plus.columns = ['bowler','5+_wkts']


bowler_stats = bowler_stats.merge(_3plus, on='bowler', how='left')
bowler_stats = bowler_stats.merge(_4plus, on='bowler', how='left')
bowler_stats = bowler_stats.merge(_5plus, on='bowler', how='left')

bowler_stats[['3+_wkts','4+_wkts','5+_wkts']] = \
                    bowler_stats[['3+_wkts','4+_wkts','5+_wkts']].fillna(0).astype(int)
#########ADDING 3+,4+,5+ wkts#########
"""
#########ADDING BEST PERFORMANCE##############
mbm_ball_bat = df_all.groupby(['bowler','match_id'])[['runs_conceeded','islegal','isBowlerWicket']]\
                                    .sum().reset_index().sort_values(by='bowler')

mbm_ball_bat = mbm_ball_bat.rename(columns={'bowler':'player'})


mbm_ball_bat['isBowlerWicket'] = mbm_ball_bat['isBowlerWicket'].fillna(0).astype(int)

# Get the index of the highest score for each striker (resolving ties using fewer balls)
idx = mbm_ball_bat.groupby('player')['isBowlerWicket'].idxmax()

# Filter the rows
most_wkts = mbm_ball_bat.loc[idx]

most_wkts['best_performance'] = most_wkts['isBowlerWicket'].astype(str)+ str('-')+\
            most_wkts['runs_conceeded'].astype(str)+str(' (')+most_wkts['islegal'].astype(str)+str(')')
        

most_wkts = most_wkts.rename(columns={'player':'bowler'})

bowler_stats = bowler_stats.merge(most_wkts[['bowler','best_performance']], on='bowler',how='left')
#########ADDING BEST PERFORMANCE##############
"""

# Sort DataFrame based on the custom order
bowler_stats = bowler_stats.sort_values(['wkts','economy'], ascending=[False, True]).reset_index(drop=True)

#batting stats

batter_stats = df_all.groupby(['striker','batting_team']).agg(  ##,'innings'
    num_innings = ('match_id','nunique'),
    runs = ('runs_off_bat','sum'),
    balls = ('is_faced_by_batter' ,'sum'),
    outs = ('is_striker_Out','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum')
    
    
    
    
).reset_index()

##addition of runouts
runout_count = df_all[df_all.wicket_type=='runout'].groupby('player_dismissed')['isWicket'].count().reset_index()

for index,row in batter_stats.iterrows():
    outs = row['outs']
    batter = row['striker']
    runout_add = 0
    
    runouts_add_df = runout_count[runout_count.player_dismissed==batter]
    if runouts_add_df.shape[0]>0:
        runout_add = runouts_add_df['isWicket'].unique()[0]
        
    outs = outs+runout_add
    batter_stats.at[index, 'outs'] = outs
#############    

############ STRIKER RUNOUT MINUS
own_runout_count = df_all[df_all.wicket_type=='runout'].groupby('player_dismissed')['is_striker_Out'].sum().reset_index()

for index,row in batter_stats.iterrows():
    outs = row['outs']
    batter = row['striker']
    own_runout_minus = 0
    
    runouts_add_df = own_runout_count[own_runout_count.player_dismissed==batter]
    if runouts_add_df.shape[0]>0:
        own_runout_minus = runouts_add_df['is_striker_Out'].unique()[0]
        
    outs = outs-own_runout_minus
    batter_stats.at[index, 'outs'] = outs
#############    
    
batter_stats['strike_rate'] = 100*batter_stats['runs']/batter_stats['balls']
batter_stats['balls_per_dismissal'] = batter_stats['balls']/batter_stats['outs']
batter_stats['bat_avg'] = batter_stats['runs']/batter_stats['outs']
batter_stats['bpb'] = batter_stats['balls']/(batter_stats['fours']+batter_stats['sixes'])
batter_stats['dot_%'] = 100*batter_stats['dots']/batter_stats['balls']

batter_stats = batter_stats.sort_values(['runs','strike_rate'], ascending=[False,False]).reset_index(drop=True)

#########ADDING 30+,50+,100+ scores#########
mbm_bat = df_all.groupby(['striker','match_id'])['runs_off_bat'].sum().reset_index().sort_values(by='striker')

_30plus = mbm_bat[mbm_bat.runs_off_bat>=30].groupby('striker')['match_id'].count().reset_index()
_50plus = mbm_bat[mbm_bat.runs_off_bat>=50].groupby('striker')['match_id'].count().reset_index()
_100plus = mbm_bat[mbm_bat.runs_off_bat>=100].groupby('striker')['match_id'].count().reset_index()

_30plus.columns = ['striker','30+_scores']
_50plus.columns = ['striker','50+_scores']
_100plus.columns = ['striker','100+_scores']


batter_stats = batter_stats.merge(_30plus, on='striker', how='left')
batter_stats = batter_stats.merge(_50plus, on='striker', how='left')
batter_stats = batter_stats.merge(_100plus, on='striker', how='left')

batter_stats[['30+_scores','50+_scores','100+_scores']] = \
                    batter_stats[['30+_scores','50+_scores','100+_scores']].fillna(0).astype(int)
#########ADDING 30+,50+,100+ scores#########
"""
#########ADDING HIGHEST SCORES##############
mbm_bat_ball = df_all.groupby(['striker','match_id'])[['runs_off_bat','is_faced_by_batter']]\
                                    .sum().reset_index().sort_values(by='striker')

mbm_bat_ball = mbm_bat_ball.rename(columns={'striker':'player'})

mbm_out = df_all.groupby(['player_dismissed','match_id'])['start_date']\
                                    .count().reset_index().sort_values(by='player_dismissed')

mbm_out = mbm_out.rename(columns={'player_dismissed':'player','start_date':'out'})
mbm_out['out'] = mbm_out['out'].fillna(0).astype(int)

mbm = mbm_bat_ball.merge(mbm_out, on=['player','match_id'], how='left')
mbm['out'] = mbm['out'].fillna(0).astype(int)

mbm['out_str'] = ''
mbm.loc[mbm['out']==0,'out_str'] = '*'

# Get the index of the highest score for each striker (resolving ties using fewer balls)
idx = mbm.groupby('player')['runs_off_bat'].idxmax()

# Filter the rows
highest_scores = mbm.loc[idx]

highest_scores['high_score'] = highest_scores['runs_off_bat'].astype(str)+ highest_scores['out_str']+ str('(')+\
                                    highest_scores['is_faced_by_batter'].astype(str)+str(')')
        

highest_scores = highest_scores.rename(columns={'player':'striker'})

batter_stats = batter_stats.merge(highest_scores[['striker','high_score']], on='striker',how='left')
#########ADDING HIGHEST SCORES##############
"""

batter_stats = batter_stats.round(2)
bowler_stats = bowler_stats.round(2)

#################                                  
df_match_info = pd.DataFrame()

df_match_info['match_id'] = df_all.match_id.unique()
df_match_info['team_1'] = ''
df_match_info['team_2'] = ''
df_match_info['team_1_total'] = 0
df_match_info['team_1_balls'] = 0
df_match_info['team_2_total'] = 0
df_match_info['team_2_balls'] = 0
df_match_info['winner'] = ''


for index,row in df_match_info.iterrows():
    subset_df = df_all[df_all.match_id==row['match_id']][df_all.innings==1]
    t1 = subset_df['batting_team'].unique()[0]
    t1_runs = subset_df['runs_scored'].max()
    t1_balls = subset_df['legal_balls_bowled'].max()
    team_1_total = t1_runs
    team_1_balls = t1_balls
    
    subset_df = df_all[df_all.match_id==row['match_id']][df_all.innings==2]
    t2 = subset_df['batting_team'].unique()[0]
    t2_runs = subset_df['runs_scored'].max()
    t2_balls = subset_df['legal_balls_bowled'].max()
    team_2_total = t2_runs
    team_2_balls = t2_balls
    
    winner = t1 if team_1_total>team_2_total else t2 if team_1_total<team_2_total else 'TIE'
    
    df_match_info.at[index, 'team_1'] = t1
    df_match_info.at[index, 'team_2'] = t2
    df_match_info.at[index, 'team_1_total'] = team_1_total
    df_match_info.at[index, 'team_1_balls'] = team_1_balls
    df_match_info.at[index, 'team_2_total'] = team_2_total
    df_match_info.at[index, 'team_2_balls'] = team_2_balls
    df_match_info.at[index, 'winner'] = winner
    


pts_table = pd.DataFrame(df_match_info.groupby('winner')['team_2_balls'].count()*2).reset_index()

pts_table.columns = ['team','points']


CSK_runs_for = []
CSK_balls_for = []
DC_runs_for = []
DC_balls_for = []
GT_runs_for = []
GT_balls_for = []
KKR_runs_for = []
KKR_balls_for = []
LSG_runs_for = []
LSG_balls_for = []
MI_runs_for = []
MI_balls_for = []
PBKS_runs_for = []
PBKS_balls_for = []
RCB_runs_for = []
RCB_balls_for = []
RR_runs_for = []
RR_balls_for = []
SRH_runs_for = []
SRH_balls_for = []

CSK_runs_against = []
CSK_balls_against = []
DC_runs_against = []
DC_balls_against = []
GT_runs_against = []
GT_balls_against = []
KKR_runs_against = []
KKR_balls_against = []
LSG_runs_against = []
LSG_balls_against = []
MI_runs_against = []
MI_balls_against = []
PBKS_runs_against = []
PBKS_balls_against = []
RCB_runs_against = []
RCB_balls_against = []
RR_runs_against = []
RR_balls_against = []
SRH_runs_against = []
SRH_balls_against = []

for m in df_all.match_id.unique():
    m_df = df_all[df_all.match_id==m].reset_index(drop=True)
    
    m_df_1 = m_df[m_df.innings==1]
    runs_1 = m_df_1.runs_scored.max()
    balls_1 = m_df_1.legal_balls_bowled.max()
    balls_1 = max(120,balls_1)
    
    m_df_2 = m_df[m_df.innings==2]
    runs_2 = m_df_2.runs_scored.max()
    balls_2 = m_df_2.legal_balls_bowled.max()
    wkts_2 = m_df_2.wickets_down.max()
    balls_2 = 120 if wkts_2==10 else balls_2
    
    
        

    if 'CSK' in m_df_1.batting_team.unique():
        CSK_runs_for.append(runs_1)
        CSK_balls_for.append(balls_1)
    if 'DC' in m_df_1.batting_team.unique():
        DC_runs_for.append(runs_1)
        DC_balls_for.append(balls_1)
    if 'GT' in m_df_1.batting_team.unique():
        GT_runs_for.append(runs_1)
        GT_balls_for.append(balls_1)
    if 'KKR' in m_df_1.batting_team.unique():
        KKR_runs_for.append(runs_1)
        KKR_balls_for.append(balls_1)
    if 'LSG' in m_df_1.batting_team.unique():
        LSG_runs_for.append(runs_1)
        LSG_balls_for.append(balls_1)
    if 'MI' in m_df_1.batting_team.unique():
        MI_runs_for.append(runs_1)
        MI_balls_for.append(balls_1)
    if 'PBKS' in m_df_1.batting_team.unique():
        PBKS_runs_for.append(runs_1)
        PBKS_balls_for.append(balls_1)
    if 'RCB' in m_df_1.batting_team.unique():
        RCB_runs_for.append(runs_1)
        RCB_balls_for.append(balls_1)
    if 'RR' in m_df_1.batting_team.unique():
        RR_runs_for.append(runs_1)
        RR_balls_for.append(balls_1)
    if 'SRH' in m_df_1.batting_team.unique():
        SRH_runs_for.append(runs_1)
        SRH_balls_for.append(balls_1)


    if 'CSK' in m_df_2.batting_team.unique():
        CSK_runs_for.append(runs_2)
        CSK_balls_for.append(balls_2)
    if 'DC' in m_df_2.batting_team.unique():
        DC_runs_for.append(runs_2)
        DC_balls_for.append(balls_2)
    if 'GT' in m_df_2.batting_team.unique():
        GT_runs_for.append(runs_2)
        GT_balls_for.append(balls_2)
    if 'KKR' in m_df_2.batting_team.unique():
        KKR_runs_for.append(runs_2)
        KKR_balls_for.append(balls_2)
    if 'LSG' in m_df_2.batting_team.unique():
        LSG_runs_for.append(runs_2)
        LSG_balls_for.append(balls_2)
    if 'MI' in m_df_2.batting_team.unique():
        MI_runs_for.append(runs_2)
        MI_balls_for.append(balls_2)
    if 'PBKS' in m_df_2.batting_team.unique():
        PBKS_runs_for.append(runs_2)
        PBKS_balls_for.append(balls_2)
    if 'RCB' in m_df_2.batting_team.unique():
        RCB_runs_for.append(runs_2)
        RCB_balls_for.append(balls_2)
    if 'RR' in m_df_2.batting_team.unique():
        RR_runs_for.append(runs_2)
        RR_balls_for.append(balls_2)
    if 'SRH' in m_df_2.batting_team.unique():
        SRH_runs_for.append(runs_2)
        SRH_balls_for.append(balls_2)

    if 'CSK' in m_df_1.bowling_team.unique():
        CSK_runs_against.append(runs_1)
        CSK_balls_against.append(balls_1)
    if 'DC' in m_df_1.bowling_team.unique():
        DC_runs_against.append(runs_1)
        DC_balls_against.append(balls_1)
    if 'GT' in m_df_1.bowling_team.unique():
        GT_runs_against.append(runs_1)
        GT_balls_against.append(balls_1)
    if 'KKR' in m_df_1.bowling_team.unique():
        KKR_runs_against.append(runs_1)
        KKR_balls_against.append(balls_1)
    if 'LSG' in m_df_1.bowling_team.unique():
        LSG_runs_against.append(runs_1)
        LSG_balls_against.append(balls_1)
    if 'MI' in m_df_1.bowling_team.unique():
        MI_runs_against.append(runs_1)
        MI_balls_against.append(balls_1)
    if 'PBKS' in m_df_1.bowling_team.unique():
        PBKS_runs_against.append(runs_1)
        PBKS_balls_against.append(balls_1)
    if 'RCB' in m_df_1.bowling_team.unique():
        RCB_runs_against.append(runs_1)
        RCB_balls_against.append(balls_1)
    if 'RR' in m_df_1.bowling_team.unique():
        RR_runs_against.append(runs_1)
        RR_balls_against.append(balls_1)
    if 'SRH' in m_df_1.bowling_team.unique():
        SRH_runs_against.append(runs_1)
        SRH_balls_against.append(balls_1)


    if 'CSK' in m_df_2.bowling_team.unique():
        CSK_runs_against.append(runs_2)
        CSK_balls_against.append(balls_2)
    if 'DC' in m_df_2.bowling_team.unique():
        DC_runs_against.append(runs_2)
        DC_balls_against.append(balls_2)
    if 'GT' in m_df_2.bowling_team.unique():
        GT_runs_against.append(runs_2)
        GT_balls_against.append(balls_2)
    if 'KKR' in m_df_2.bowling_team.unique():
        KKR_runs_against.append(runs_2)
        KKR_balls_against.append(balls_2)
    if 'LSG' in m_df_2.bowling_team.unique():
        LSG_runs_against.append(runs_2)
        LSG_balls_against.append(balls_2)
    if 'MI' in m_df_2.bowling_team.unique():
        MI_runs_against.append(runs_2)
        MI_balls_against.append(balls_2)
    if 'PBKS' in m_df_2.bowling_team.unique():
        PBKS_runs_against.append(runs_2)
        PBKS_balls_against.append(balls_2)
    if 'RCB' in m_df_2.bowling_team.unique():
        RCB_runs_against.append(runs_2)
        RCB_balls_against.append(balls_2)
    if 'RR' in m_df_2.bowling_team.unique():
        RR_runs_against.append(runs_2)
        RR_balls_against.append(balls_2)
    if 'SRH' in m_df_2.bowling_team.unique():
        SRH_runs_against.append(runs_2)
        SRH_balls_against.append(balls_2)

CSK_runs_for = sum(CSK_runs_for)
CSK_balls_for = sum(CSK_balls_for)
CSK_runs_against = sum(CSK_runs_against)
CSK_balls_against = sum(CSK_balls_against)
DC_runs_for = sum(DC_runs_for)
DC_balls_for = sum(DC_balls_for)
DC_runs_against = sum(DC_runs_against)
DC_balls_against = sum(DC_balls_against)
GT_runs_for = sum(GT_runs_for)
GT_balls_for = sum(GT_balls_for)
GT_runs_against = sum(GT_runs_against)
GT_balls_against = sum(GT_balls_against)
KKR_runs_for = sum(KKR_runs_for)
KKR_balls_for = sum(KKR_balls_for)
KKR_runs_against = sum(KKR_runs_against)
KKR_balls_against = sum(KKR_balls_against)
LSG_runs_for = sum(LSG_runs_for)
LSG_balls_for = sum(LSG_balls_for)
LSG_runs_against = sum(LSG_runs_against)
LSG_balls_against = sum(LSG_balls_against)
MI_runs_for = sum(MI_runs_for)
MI_balls_for = sum(MI_balls_for)
MI_runs_against = sum(MI_runs_against)
MI_balls_against = sum(MI_balls_against)
PBKS_runs_for = sum(PBKS_runs_for)
PBKS_balls_for = sum(PBKS_balls_for)
PBKS_runs_against = sum(PBKS_runs_against)
PBKS_balls_against = sum(PBKS_balls_against)
RCB_runs_for = sum(RCB_runs_for)
RCB_balls_for = sum(RCB_balls_for)
RCB_runs_against = sum(RCB_runs_against)
RCB_balls_against = sum(RCB_balls_against)
RR_runs_for = sum(RR_runs_for)
RR_balls_for = sum(RR_balls_for)
RR_runs_against = sum(RR_runs_against)
RR_balls_against = sum(RR_balls_against)
SRH_runs_for = sum(SRH_runs_for)
SRH_balls_for = sum(SRH_balls_for)
SRH_runs_against = sum(SRH_runs_against)
SRH_balls_against = sum(SRH_balls_against)


teams = ["CSK", "DC", "GT","KKR","LSG","MI","PBKS","RCB","RR","SRH"]
pts_table_p2 = pd.DataFrame()
pts_table_p2['team'] = np.array(teams)
pts_table_p2['matches_played'] = 0

for index, row in pts_table_p2.iterrows():
    team = row['team']
    matches_played = df_all[df_all.batting_team==team].match_id.nunique()
    
    pts_table_p2.at[index, 'matches_played'] = matches_played

runs_for = pd.Series([CSK_runs_for, DC_runs_for, GT_runs_for, KKR_runs_for, LSG_runs_for,
           MI_runs_for, PBKS_runs_for, RCB_runs_for, RR_runs_for, SRH_runs_for])

balls_for = pd.Series([CSK_balls_for, DC_balls_for, GT_balls_for, KKR_balls_for, LSG_balls_for,
           MI_balls_for, PBKS_balls_for, RCB_balls_for, RR_balls_for, SRH_balls_for])

runs_against = pd.Series([CSK_runs_against, DC_runs_against, GT_runs_against, KKR_runs_against, LSG_runs_against,
           MI_runs_against, PBKS_runs_against, RCB_runs_against, RR_runs_against, SRH_runs_against])

balls_against = pd.Series([CSK_balls_against, DC_balls_against, GT_balls_against, KKR_balls_against, LSG_balls_against,
           MI_balls_against, PBKS_balls_against, RCB_balls_against, RR_balls_against, SRH_balls_against])

pts_table_p2['runs_for'] = runs_for
pts_table_p2['balls_for'] = balls_for

pts_table_p2['runs_against'] = runs_against
pts_table_p2['balls_against'] = balls_against

pts_table_p2['NRR'] = (6*pts_table_p2['runs_for']/pts_table_p2['balls_for']) - \
                                6*pts_table_p2['runs_against']/pts_table_p2['balls_against']


pts_table = pd.merge(left=pts_table, right=pts_table_p2, on='team', how='outer')
pts_table.fillna(0, inplace=True)
pts_table['points'] = pts_table['points'].astype(int)

pts_table = pts_table.sort_values(by=['points','NRR'], ascending=False).reset_index(drop=True)

# In[ ]:






/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_17264/384800036.py:205: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_df = df_all[df_all.match_id==row['match_id']][df_all.innings==1]
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_17264/384800036.py:212: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_df = df_all[df_all.match_id==row['match_id']][df_all.innings==2]


pts_table.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/points_table.csv', index=None)

In [11]:
batter_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/batter_stats.csv', index=None)
bowler_stats.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/bowler_stats.csv', index=None)

In [12]:
df_match_info.to_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/df_match_info.csv', index=None)